# Creating an Interactive Dashboard using `hvPlot` and `Panel`

---

In [ ]:
import xarray as xr
import holoviews as hv
import panel as pn
import hvplot.xarray
from holoviews import opts

pn.extension()


## Data

In this notebook, we are going to load annual mean dataset of 2-m temperature from the [ERA5 reanalysis](https://doi.org/10.24381/cds.adbb2d47) that we preprocessed in Zarr format. Please see the preprocessing notebooks for the required steps.

In [ ]:
rda_url = "https://data.rda.ucar.edu/"
annual_means = rda_url + "pythia_era5_24/annual_means/"
xrds = xr.open_dataset(annual_means + "temp_2m_annual_1940_2023.zarr", engine="zarr")
xrds

In [ ]:
# Select the time range of interest
xrds = xrds.sel(time=slice('2017-01-01', '2023-12-31'))
xrds.load();

## Panel Widgets

Panel provides a variety of widgets that can be used to build interactive dashboards. In this notebook, we are going to use some of these widgets. For the complete list of widgets, please see the [Panel documentation](https://panel.holoviz.org/api/panel.widgets.html).

The panel widgets that we are using are: 
- `pn.widgets.Select` for selecting the variable
- `pn.widgets.DatePicker` for selecting the date
- `pn.widgets.Player` for making time series animations

In [ ]:

w_var = pn.widgets.Select(name="Data Variable", options=list(xrds.data_vars))

dataset_controls = pn.WidgetBox(
    "## Dataset Controls",
    w_var,
)
dataset_controls

Now, let's create dropdown widgets for selecting the colormap and plot type.  

In [ ]:
w_cmap = pn.widgets.Select(name="Colormap", options=["inferno", "plasma", "coolwarm"])


w_plot_type = pn.widgets.Select(
    name="Plot Type", options=["Color Plot", "Contour"]
)


plot_controls = pn.WidgetBox(
    "## Plot Controls",
    w_plot_type,
    w_cmap,
)
plot_controls

Now, let's put together all the controls and the plot in a panel layout using `pn.Row` and `pn.Column`:

In [ ]:
controls = pn.Column(dataset_controls, plot_controls)
controls

In [ ]:
w_player = pn.widgets.Player(
    value=0,
    start=0,
    end=len(xrds.time) - 1,
    name="Year",
    loop_policy="loop",
    interval=300,
    align="center",
    width_policy="fit",
)
w_player

## Plotting Function

In [ ]:
def plot_ds(time, var, cmap, plot_type):
    clim = (xrds[var].values.min(), xrds[var].values.max())

    if plot_type == "Color Plot":
        return (
            xrds[var]
            .isel(time=time)
            .hvplot(
                cmap=cmap,
                title=str(f"{var} year {time}"),
                clim=clim,
                dynamic=False,
                rasterize=True,
                precompute=True,
            )
            .opts(framewise=False)
        )

    elif plot_type == "Contour":
        return (
            xrds[var]
            .isel(time=time)
            .hvplot.contour(
                cmap=cmap,
                dynamic=False,
                rasterize=True,
                title=str(f"{var} Year: {time}"),
                clim=clim,
                precompute=True,
            )
            .opts(framewise=False)
        )

In [ ]:

app = pn.Row(
    controls,
    pn.Column(
        pn.panel(
            hv.DynamicMap(
                pn.bind(
                    plot_ds,
                    time=w_player,
                    var=w_var,
                    cmap=w_cmap,
                    plot_type=w_plot_type,
                )
            )
        ),
        w_player,
    ),
).servable()

app

Please note how the above dashboard is servable. You can deploy the dashboard by running the following command in the terminal:

```bash
panel serve --show 04_dashboard.ipynb --allow-websocket-origin=projectpythia.2i2c.cloud
```

This will open a new tab in your default web browser with the dashboard. The allow websocket origin flag is required to allow traffic to flow to the site. This should be update to reflect the base URL where the application is launched. A wildcard can be used, `*`, to allow traffic from any site to connect. 

On the Project Pythia 2i2c hosted JupyterHub the link to use to access the panel serve command is [https://projectpythia.2i2c.cloud/hub/user-redirect/proxy/5006/04_dashboard](https://projectpythia.2i2c.cloud/hub/user-redirect/proxy/5006/04_dashboard)